# 神经网络风格迁移 ( 英语：Neural Style Transfer)

<br></br>

 * [在 Google Colab 运行](https://colab.research.google.com/github/dslab-deepflying/deepflying/blob/master/InnoCamp/NST/nst.ipynb)
 * [浏览 GitHub 源码 ](https://github.com/dslab-deepflying/deepflying/blob/master/InnoCamp/NST/nst.ipynb)


## 概述

神经风格迁移是今年来深度学习在机器学习领域中相当成功的应用，它最初的想法来自这篇论文[ Leon A. Gatys' paper, A Neural Algorithm of Artistic Style](https://arxiv.org/abs/1508.06576)，它不仅改变了传统计算机视觉领域对图像分析的手动建模方式，还创造性的将图片的风格和内容直观的分别开来，便于生成各式各样有趣的图片。

## 环境搭建

In [ ]:
from __future__ import print_function
from keras.preprocessing.image import load_img, save_img, img_to_array
import numpy as np
from scipy.optimize import fmin_l_bfgs_b
import time
import cv2

import matplotlib.pyplot as plt
%matplotlib inline

from keras.applications import vgg19
from keras import backend as K

import os
# 显卡编号 可选 0 或 1
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import tensorflow as tf 
from keras.backend.tensorflow_backend import set_session 

# 显卡使用率 方便大家共同使用请勿更改此数值
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.20) 
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

### 所需包的导入

<br>

包（英文：package）是 python 中对一系列方法数据的封装，有面向对象编程（英文：Obect Oriented Coding，简称 OO）经验的同学来说并不陌生。对于没有OO基础的同学来说，包 可以简单的；理解为一种类似 “工具箱” 的概念。

<br>

我们本次实验使用的核心的包是[TensorFlow](https://www.tensorflow.org/?hl=zh_cn) ，谷歌公司的机器学习框架，也是当今世界上应用最广，发展最迅速的机器学习框架。[Keras](https://keras.io/zh/)是基于TensorFlow框架设计的高层抽象机器学习包。它使得我们能够略过数据操作中的张量操作以及计算图的编写运行等相对底层的操作，通过简单的 序贯模型 和 函数模型 快速搭建需要的机器学习模型。

<br>

其他包：<br>
1）scipy ， python的算法库和数据工具包<br>
2）numpy ，python的数据运算工具包<br>
3）matplotlib ， python用于图像表格绘制的包<br>


In [ ]:
import os,sys

if not os.path.exists('data'):
    os.makedirs('data') 
    
  
# !wget -q https://github.com/JChrysanthemum/deepflying/raw/master/InnoCamp/NST/pics/pic.zip -O data/pic.zip
# !unzip -o data/pic.zip -d data

print("--- 内容图片 ---")

c1 = load_img("data/c1.png")
c1 = img_to_array(c1)
c1 = np.array(c1, np.uint8)

c2 = load_img("data/c2.jpg")
c2 = img_to_array(c2)
c2 = np.array(c2, np.uint8)

c3 = load_img("data/c3.jpg")
c3 = img_to_array(c3)
c3 = np.array(c3, np.uint8)

c4 = load_img("data/c4.jpg")
c4 = img_to_array(c4)
c4 = np.array(c4, np.uint8)


plt.figure(figsize=(18, 12))

plt.subplot(2, 3, 1), plt.title('model')
plt.axis('off')
plt.imshow(c1)

plt.subplot(2, 3, 2), plt.title('chang-e')
plt.axis('off')
plt.imshow(c2)

plt.subplot(2, 3, 3), plt.title('violin')
plt.axis('off')
plt.imshow(c3)

plt.subplot(2, 3, 4), plt.title('deadpool')
plt.axis('off')
plt.imshow(c4)
plt.show()


print("--- 风格图片 ---")

s1 = load_img("data/s1.jpg")
s1 = img_to_array(s1)
s1 = np.array(s1, np.uint8)

s2 = load_img("data/s2.jpg")
s2 = img_to_array(s2)
s2 = np.array(s2, np.uint8)

s3 = load_img("data/s3.jpg")
s3 = img_to_array(s3)
s3 = np.array(s3, np.uint8)

s4 = load_img("data/s4.jpg")
s4 = img_to_array(s4)
s4 = np.array(s4, np.uint8)


plt.figure(figsize=(18, 12))

plt.subplot(2, 3, 1), plt.title('PuSa')
plt.axis('off')
plt.imshow(s1)

plt.subplot(2, 3, 2), plt.title('Bird')
plt.axis('off')
plt.imshow(s2)

plt.subplot(2, 3, 3), plt.title('Art1')
plt.axis('off')
plt.imshow(s3)

plt.subplot(2, 3, 4), plt.title('Art2')
plt.axis('off')
plt.imshow(s4)



plt.show()

### 所需文件下载

在这次实验中，我们用到三张（也可以说是三种）图片。

1）内容图片，本次默认使用的为data/c1.png，是2017年巴黎时装周的一张图片。

2）风格图片，本次默认使用的为data/s1.jpg，是敦煌某张壁画中的一小块。

3）掩膜图片，本次默认使用的为data/m1.png，是内容图片中除服饰外的图片部分。用到的深度抠图（deepmatting）也是深度学习的一个分支。

<br>

以及VGG19神经网络，原用于物体识别。

## 代码编写

接下来，我们还动手实现神经风格迁移。

### 超参数设置以及变量配置

超参数是决定整个AI模型训练结果的 全局参数。在开头设置方便大家随时调整观察不同的结果。

### 图片预处理

In [ ]:
#@markdown <b>参数以及变量配置 <br><br>

#@markdown 内容图片 （默认：女模特 ）：
base_image_path = "model" #@param ["model","chang-e","violin","deadpool"]

#@markdown 风格图片 （默认：菩萨 ）：
style_reference_image_path = "Art1" #@param ["PuSa","Bird","Art1","Art2"]

if base_image_path == "model":
  base_image_path = "data/c1.png"
  mask_image_path = "data/m1.png"
elif base_image_path == "chang-e":
  base_image_path = "data/c2.jpg"
  mask_image_path = "data/m2.jpg"
elif base_image_path == "violin":
  base_image_path = "data/c3.jpg"
  mask_image_path = "data/m3.jpg"
elif base_image_path == "deadpool":
  base_image_path = "data/c4.jpg"
  mask_image_path = "data/m4.jpg"
  
  
if style_reference_image_path == "PuSa" : 
  style_reference_image_path = "data/s1.jpg"
elif style_reference_image_path == "Bird" : 
  style_reference_image_path = "data/s2.jpg"
elif style_reference_image_path == "Art1" : 
  style_reference_image_path = "data/s3.jpg"
elif style_reference_image_path == "Art2" : 
  style_reference_image_path = "data/s4.jpg"

result_prefix = "combined"


#@markdown 迭代次数 ：
iterations = 50 #@param {type:"slider", min:10, max:200, step:5}
# these are the weights of the different loss components
#@markdown 总异变权重（默认0.025）：
total_variation_weight = 0.025 #@param {type:"slider", min:0, max:0.08, step:0.025}
#@markdown 风格权重（默认1）：
style_weight = 1 #@param {type:"slider", min:0.4, max:1.2, step:0.1}
#@markdown 内容权重（默认1）：
content_weight = 1 #@param {type:"slider", min:0.4, max:1.2, step:0.1}
# 保存图片的间隔数
#@markdown 图片保存和显示间隔 （默认 5 ）：
SAVE_INTERVAL = 5 #@param {type:"slider", min:0, max:2000, step:5}

# dimensions of the generated picture.
width, height = load_img(base_image_path).size

#@markdown 图片缩放后的宽度 （默认 300 像素 ）：
img_nrows = 300 #@param {type:"slider", min:100, max:400, step:50}
img_ncols = int(width * img_nrows / height)




  

In [ ]:
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(img_nrows, img_ncols))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = vgg19.preprocess_input(img)
    return img

# util function to convert a tensor into a valid image

### 图片解码

In [ ]:
def deprocess_image(x):
    if K.image_data_format() == 'channels_first':
        x = x.reshape((3, img_nrows, img_ncols))
        x = x.transpose((1, 2, 0))
    else:
        x = x.reshape((img_nrows, img_ncols, 3))
    # Remove zero-center by mean pixel
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    # 'BGR'->'RGB'
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x

# get tensor representations of our images

### 模型构建

In [ ]:
base_image = K.variable(preprocess_image(base_image_path))
style_reference_image = K.variable(preprocess_image(style_reference_image_path))

# this will contain our generated image
if K.image_data_format() == 'channels_first':
    combination_image = K.placeholder((1, 3, img_nrows, img_ncols))
else:
    combination_image = K.placeholder((1, img_nrows, img_ncols, 3))

# combine the 3 images into a single Keras tensor
input_tensor = K.concatenate([base_image,
                              style_reference_image,
                              combination_image], axis=0)

# build the VGG19 network with our 3 images as input
# the model will be loaded with pre-trained ImageNet weights
model = vgg19.VGG19(input_tensor=input_tensor,
                    weights='imagenet', include_top=False)
print('Model loaded.')

# get the symbolic outputs of each "key" layer (we gave them unique names).
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])

# compute the neural style loss
# first we need to define 4 util functions

# the gram matrix of an image tensor (feature-wise outer product)

### 定义gram矩阵

In [ ]:
def gram_matrix(x):
    assert K.ndim(x) == 3
    if K.image_data_format() == 'channels_first':
        features = K.batch_flatten(x)
    else:
        features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
    gram = K.dot(features, K.transpose(features))
    return gram

### 定义风格损失

In [ ]:
def style_loss(style, combination):
    assert K.ndim(style) == 3
    assert K.ndim(combination) == 3
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = img_nrows * img_ncols
    return K.sum(K.square(S - C)) / (4.0 * (channels ** 2) * (size ** 2))

### 定义内容损失

In [ ]:
def content_loss(base, combination):
    return K.sum(K.square(combination - base))

### 定义异变损失


In [ ]:
def total_variation_loss(x):
    assert K.ndim(x) == 4
    if K.image_data_format() == 'channels_first':
        a = K.square(
            x[:, :, :img_nrows - 1, :img_ncols - 1] - x[:, :, 1:, :img_ncols - 1])
        b = K.square(
            x[:, :, :img_nrows - 1, :img_ncols - 1] - x[:, :, :img_nrows - 1, 1:])
    else:
        a = K.square(
            x[:, :img_nrows - 1, :img_ncols - 1, :] - x[:, 1:, :img_ncols - 1, :])
        b = K.square(
            x[:, :img_nrows - 1, :img_ncols - 1, :] - x[:, :img_nrows - 1, 1:, :])
    return K.sum(K.pow(a + b, 1.25))

### 损失计算方法和模型层选定

In [ ]:
loss = K.variable(0.0)
layer_features = outputs_dict['block5_conv2']
base_image_features = layer_features[0, :, :, :]
combination_features = layer_features[2, :, :, :]
loss += content_weight * content_loss(base_image_features,
                                      combination_features)

feature_layers = ['block1_conv1', 'block2_conv1',
                  'block3_conv1', 'block4_conv1',
                  'block5_conv1']
for layer_name in feature_layers:
    layer_features = outputs_dict[layer_name]
    style_reference_features = layer_features[1, :, :, :]
    combination_features = layer_features[2, :, :, :]
    sl = style_loss(style_reference_features, combination_features)
    loss += (style_weight / len(feature_layers)) * sl
loss += total_variation_weight * total_variation_loss(combination_image)

# get the gradients of the generated image wrt the loss
grads = K.gradients(loss, combination_image)

outputs = [loss]
if isinstance(grads, (list, tuple)):
    outputs += grads
else:
    outputs.append(grads)

f_outputs = K.function([combination_image], outputs)

### 定义梯度评价方法



In [ ]:
def eval_loss_and_grads(x):
    if K.image_data_format() == 'channels_first':
        x = x.reshape((1, 3, img_nrows, img_ncols))
    else:
        x = x.reshape((1, img_nrows, img_ncols, 3))
    outs = f_outputs([x])
    loss_value = outs[0]
    if len(outs[1:]) == 1:
        grad_values = outs[1].flatten().astype('float64')
    else:
        grad_values = np.array(outs[1:]).flatten().astype('float64')
    return loss_value, grad_values

### 定义评价器

In [ ]:
class Evaluator(object):

    def __init__(self):
        self.loss_value = None
        self.grads_values = None

    def loss(self, x):
        assert self.loss_value is None
        loss_value, grad_values = eval_loss_and_grads(x)
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value

    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values

## 开始训练

In [ ]:
def train():
    evaluator = Evaluator()

    # run scipy-based optimization (L-BFGS) over the pixels of the generated image
    # so as to minimize the neural style loss
    x = preprocess_image(base_image_path)

    mask = load_img(mask_image_path, target_size=(img_nrows, img_ncols))
    mask = img_to_array(mask)[:, :, 0]
    mask = np.array(mask, np.uint8)

    ori = load_img(base_image_path, target_size=(img_nrows, img_ncols))
    ori = img_to_array(ori)
    ori = np.array(ori, np.uint8)

    sty = load_img(style_reference_image_path, target_size=(img_nrows, img_ncols))
    sty = img_to_array(sty)
    sty = np.array(sty, np.uint8)

    plt.figure(figsize=(5, 6))
    plt.subplot(2, 3, 1), plt.title('Content')
    plt.axis('off')
    plt.imshow(ori)
    plt.subplot(2, 3, 2), plt.title('Style')
    plt.axis('off')
    plt.imshow(sty)
    plt.show()

    for i in range(iterations):
        start_time = time.time()
        x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x.flatten(),
                                         fprime=evaluator.grads, maxfun=20)
        end_time = time.time()

        if i % SAVE_INTERVAL == 0:
            # save current generated image
            img = deprocess_image(x.copy())
            fname = result_prefix + '_at_iteration_%d.png' % i
            save_img(fname, img)
            # end_time = time.time()

            bac_mask = cv2.bitwise_not(mask)
            bac_img = cv2.bitwise_and(ori, ori, mask=bac_mask)
            masked_img = cv2.bitwise_and(img, img, mask=mask)
            masked_img = cv2.add(bac_img, masked_img)
            print('Image saved as', fname)
            plt.imshow(masked_img)
            plt.show()

        sys.stdout.write(
            "\r Iteration %d [ loss: %f ] , time : %ds " % (i, min_val, end_time - start_time))
        sys.stdout.flush()

    print(" \n\rTranslation complete ! \n\r")
    plt.figure(figsize=(20, 24))
    plt.subplot(4, 6, 1), plt.title('Origin')
    plt.axis('off')
    plt.imshow(ori)
    plt.subplot(4, 6, 2), plt.title('NST')
    plt.axis('off')
    plt.imshow(masked_img)
    plt.show()
    
train()